In [ ]:
import os
import csv
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
import torch.optim as optim
import numpy as np
from torchvision import transforms

In [ ]:
samples=[]
classes={}
classes['forest']=0
classes['highway']=1
classes['insidecity']=2
classes['mountain']=3
classes['street']=4
sampleClasses=[]
X_tr = []; X_te = []; Y_tr = []; Y_te = [];

In [ ]:
for root, dirs, files in os.walk("/content/drive/My Drive/DEEP LEARNING ASSIGNMENTS/OUR_GROUP", topdown=False):
    for name in files:
        filepath=os.path.join(root, name)
        f=open(filepath,'r')
        data=f.read().replace('\n', ' ').split(' ')
        data = list(filter(None, data))
        samples.append(data)
        #print(root)
        sampleClass=classes[root.split('/')[-1]]
        sampleClasses.append(sampleClass)
        f.close()

KeyboardInterrupt: ignored

In [ ]:
#split
X_tr, X_te, Y_tr, Y_te = train_test_split(samples,sampleClasses,train_size = 0.8,random_state = 5)

print(np.shape(X_tr), np.shape(X_te), np.shape(Y_tr), np.shape(Y_te))
print(X_tr[0])


(28, 828) (8, 828) (28,) (8,)
['0.31061', '0.012481', '0.89813', '0.00059684', '3.623e-09', '3.2266e-07', '0.39077', '0.89995', '-0.42538', '0.00032757', '0.0032505', '0.0012599', '0.0015371', '0.00037797', '0.0027969', '0.0015623', '0.033337', '1.6503', '1.7649', '1.1946', '2.108', '1.868', '1.8585', '0.59129', '0.12915', '0.56515', '0.0018641', '2.7475e-06', '4.5444e-05', '-1.469', '-0.66722', '0.2193', '0.00047873', '0.0026456', '0.0017134', '0.0017889', '0.00012598', '0.0021165', '0.0022677', '0.03331', '2.6124', '2.8811', '1.6538', '2.3341', '2.9266', '1.826', '0.61742', '0.17145', '0.53059', '2.4062e-07', '7.1198e-06', '0.00010959', '0.357', '0.41028', '0.20211', '0.00050393', '0.0044598', '0.0027464', '0.0017134', '0.00020157', '0.0042078', '0.0034015', '0.027212', '1.7853', '2.3098', '1.9028', '2.4572', '2.7219', '1.3934', '0.56431', '0.18376', '0.51273', '3.3846e-05', '5.3535e-06', '3.6465e-06', '-0.29423', '0.026719', '-0.84047', '0.00057952', '0.0030992', '0.0021165', '0.001

In [ ]:
#batch data
trainingSamplesX = np.asarray(X_tr, dtype=np.float64)
trainingSamplesY = np.asarray(Y_tr, dtype=np.int64)

testSamplesX = np.asarray(X_te, dtype=np.float64)
testSamplesY = np.asarray(Y_te, dtype=np.int64)

traindata = torch.utils.data.TensorDataset(torch.from_numpy(trainingSamplesX).float()
                                       ,torch.from_numpy(trainingSamplesY).long())

b_train = torch.utils.data.DataLoader(traindata,batch_size = 20,shuffle = False)

testdata = torch.utils.data.TensorDataset(torch.from_numpy(testSamplesX).float()
                                       ,torch.from_numpy(testSamplesY).long())

b_test = torch.utils.data.DataLoader(testdata,batch_size = 20,shuffle = False)



In [ ]:
#class for encoder framework - 1
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.e1 = nn.Linear(828,512)
        self.e2 = nn.Linear(512,256)
        self.d1 = nn.Linear(256,512)
        self.d2 = nn.Linear(512,828)
    
    def forward(self,x):
        x = F.relu(self.e1(x))
        x = self.e2(x)
        x = F.relu(self.d1(x))
        x = self.d2(x)
        return x
    
    def redRep1(self,x):
        x = F.relu(self.e1(x))
        x = self.e2(x)
        return x
    
net1 = Net1()
print(net1)

Net1(
  (e1): Linear(in_features=828, out_features=512, bias=True)
  (e2): Linear(in_features=512, out_features=256, bias=True)
  (d1): Linear(in_features=256, out_features=512, bias=True)
  (d2): Linear(in_features=512, out_features=828, bias=True)
)


In [ ]:
optimizer1 = optim.Adam(net1.parameters(), lr = 1e-3)

In [ ]:
#ENCODER 1
prev_loss = 0;cur_loss = 10
while((abs(prev_loss-cur_loss)>1e-1)):
    prev_loss = cur_loss
    running_loss = 0
    for d in b_train:
        #print(d.shape)
        x,y = d
        net1.zero_grad()
        
        output = net1(x.view(-1,828))
        loss = F.mse_loss(output,x.view(-1,828))
        running_loss += loss
        loss.backward()
        optimizer1.step()
    print(running_loss/len(b_train))
    cur_loss = running_loss/len(b_train)
    
print('TRAINED 1ST stacking')

tensor(1.6560, grad_fn=<DivBackward0>)
tensor(0.9616, grad_fn=<DivBackward0>)
tensor(0.5446, grad_fn=<DivBackward0>)
tensor(0.2604, grad_fn=<DivBackward0>)
tensor(0.3036, grad_fn=<DivBackward0>)
TRAINED 1ST stacking


In [ ]:
#class for encoder framework - 2
class Net2(nn.Module):
    def __init__(self):
        super().__init__()
        self.e3 = nn.Linear(256,128)
        self.e4 = nn.Linear(128,128)
        self.d3 = nn.Linear(128,128)
        self.d4 = nn.Linear(128,256)
        
    def forward(self,x):
        x = F.relu(self.e3(x))
        x = self.e4(x)
        x = F.relu(self.d3(x))
        x = self.d4(x)
        return x
    
    def redRep2(self,x):
        x = F.relu(self.e3(x))
        x = self.e4(x)
        return x
    
net2 = Net2()
print(net2)

Net2(
  (e3): Linear(in_features=256, out_features=128, bias=True)
  (e4): Linear(in_features=128, out_features=128, bias=True)
  (d3): Linear(in_features=128, out_features=128, bias=True)
  (d4): Linear(in_features=128, out_features=256, bias=True)
)


In [ ]:
optimizer2 = optim.Adam(net2.parameters(), lr = 1e-3)

In [ ]:
#ENCODER 2
prev_loss = 0;cur_loss = 10
while(abs(prev_loss-cur_loss)>1e-1):
    prev_loss = cur_loss
    running_loss = 0
    for d in b_train:
        #print(d.shape)
        
        x,y = d
        net2.zero_grad()
        temp = net1.redRep1(x.view(-1,828))
        output = net2(temp)
        loss = F.mse_loss(output,temp)
        running_loss += loss
        loss.backward()
        optimizer2.step()
    print(running_loss/len(b_train))
    cur_loss = running_loss/len(b_train)
    
print('TRAINED 2ND stacking')

tensor(2.4001, grad_fn=<DivBackward0>)
tensor(2.1903, grad_fn=<DivBackward0>)
tensor(1.8258, grad_fn=<DivBackward0>)
tensor(1.2978, grad_fn=<DivBackward0>)
tensor(0.8620, grad_fn=<DivBackward0>)
tensor(0.6870, grad_fn=<DivBackward0>)
tensor(0.3910, grad_fn=<DivBackward0>)
tensor(0.1899, grad_fn=<DivBackward0>)
tensor(0.1566, grad_fn=<DivBackward0>)
TRAINED 2ND stacking


In [ ]:
#class for encoder framework - 3
class Net3(nn.Module):
    def __init__(self):
        super().__init__()
        self.e5 = nn.Linear(128,64)
        self.e6 = nn.Linear(64,64)
        self.d5 = nn.Linear(64,64)
        self.d6 = nn.Linear(64,128)
    def forward(self,x):
        x = F.relu(self.e5(x))
        x = self.e6(x)
        x = F.relu(self.d5(x))
        x = self.d6(x)
        return x
    
    def redRep3(self,x):
        x = F.relu(self.e5(x))
        x = self.e6(x)
        return x

net3 = Net3()
print(net3)

Net3(
  (e5): Linear(in_features=128, out_features=64, bias=True)
  (e6): Linear(in_features=64, out_features=64, bias=True)
  (d5): Linear(in_features=64, out_features=64, bias=True)
  (d6): Linear(in_features=64, out_features=128, bias=True)
)


In [ ]:
optimizer3 = optim.Adam(net3.parameters(), lr = 1e-3)

In [ ]:
#ENCODER 3
prev_loss = 0;cur_loss = 10
while(abs(prev_loss-cur_loss)>1e-1):
    prev_loss = cur_loss
    running_loss = 0
    for d in b_train:
        #print(d.shape)
        
        x,y = d
        net3.zero_grad()
        temp = net2.redRep2(net1.redRep1(x.view(-1,828)))
        output = net3(temp)
        loss = F.mse_loss(output,temp)
        running_loss += loss
        loss.backward()
        optimizer3.step()
    print(running_loss/len(b_train))
    cur_loss = running_loss/len(b_train)
    
print('TRAINED 3RD stacking')

tensor(2.8557, grad_fn=<DivBackward0>)
tensor(2.7734, grad_fn=<DivBackward0>)
TRAINED 3RD stacking


In [ ]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.e1 = net1.e1
        self.e2 = net1.e2
        self.e3 = net2.e3
        self.e4 = net2.e4
        self.e5 = net3.e5
        self.e6 = net3.e6
        
        self.fc1 = nn.Linear(64,100)
        self.fc2 = nn.Linear(100,5)
    
    def forward(self,x):
        x = F.relu(self.e1(x))
        x = self.e2(x)
        
        x = F.relu(self.e3(x))
        x = self.e4(x)
        
        x = F.relu(self.e5(x))
        x = self.e6(x)
        
        x = torch.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

clf = Classifier()
print(clf.parameters)

<bound method Module.parameters of Classifier(
  (e1): Linear(in_features=828, out_features=512, bias=True)
  (e2): Linear(in_features=512, out_features=256, bias=True)
  (e3): Linear(in_features=256, out_features=128, bias=True)
  (e4): Linear(in_features=128, out_features=128, bias=True)
  (e5): Linear(in_features=128, out_features=64, bias=True)
  (e6): Linear(in_features=64, out_features=64, bias=True)
  (fc1): Linear(in_features=64, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=5, bias=True)
)>


In [ ]:
optimizer4 = optim.Adam(clf.parameters(), lr = 1e-4)
criterion = nn.CrossEntropyLoss()

In [ ]:
#CLASSIFIER
prev_loss = 0;cur_loss = 10
while(abs(prev_loss-cur_loss)>1e-1):
    prev_loss = cur_loss
    running_loss = 0
    for d in b_train:
        #print(d.shape)
        
        x,y = d
        clf.zero_grad()
        output = clf(x.view(-1,828))
        loss = criterion(output,y)
        running_loss += loss
        loss.backward()
        optimizer4.step()
    print(running_loss/len(b_train))
    cur_loss = running_loss/len(b_train)
    
print('TRAINED classifier')

tensor(1.5760, grad_fn=<DivBackward0>)
tensor(1.4848, grad_fn=<DivBackward0>)
TRAINED classifier
